In [2]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense  
import pandas as pd

In [2]:
dataTrain=pd.read_csv("../data/newtrain.csv")
dataTest=pd.read_csv("../data/newtest.csv")
del dataTrain["Unnamed: 0"]
del dataTest["Unnamed: 0"]
del dataTrain["target_away"]
del dataTest["target_away"]

del dataTrain["gf_rolling"]
del dataTest["gf_rolling"]

del dataTrain["ga_rolling"]
del dataTest["ga_rolling"]

del dataTrain["sh_rolling"]
del dataTest["sh_rolling"]

del dataTrain["sot_rolling"]
del dataTest["sot_rolling"]

del dataTrain["dist_rolling"]
del dataTest["dist_rolling"]

del dataTrain["fk_rolling"]
del dataTest["fk_rolling"]

del dataTrain["pk_rolling"]
del dataTest["pk_rolling"]

del dataTrain["pkatt_rolling"]
del dataTest["pkatt_rolling"]
dataTrain

,round,venue,gf,ga,xg,xga,poss,formation,sh,sot,dist,fk,pk,pkatt,venue_code,opp_code,hour,day_code,home_code,target_home
0,33,0,0.0,0.0,1.4,0.8,49.0,4,11.0,2.0,17.5,0.0,0.0,0.0,0,5,13,6,0,0
1,35,1,2.0,0.0,2.4,0.4,64.0,4,25.0,8.0,18.4,2.0,0.0,0.0,1,4,12,6,0,1
2,37,0,5.0,2.0,3.5,1.0,67.0,4,30.0,14.0,16.6,2.0,0.0,0.0,0,7,13,6,0,1
3,2,0,1.0,3.0,0.3,1.8,43.0,4,6.0,3.0,20.5,1.0,0.0,0.0,0,12,14,6,0,0
4,3,1,2.0,0.0,2.1,0.7,43.0,4,14.0,5.0,16.4,1.0,0.0,0.0,1,17,12,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,24,0,1.0,0.0,0.5,0.9,50.0,12,7.0,3.0,14.9,1.0,0.0,0.0,0,8,20,4,19,1
1802,28,1,2.0,1.0,1.1,0.3,61.0,12,16.0,6.0,20.5,1.0,0.0,0.0,1,4,12,6,19,1
1803,29,0,0.0,2.0,0.9,3.2,38.0,12,15.0,3.0,17.6,0.0,0.0,0.0,0,17,14,6,19,0
1804,30,0,0.0,1.0,0.3,0.7,59.0,3,9.0,3.0,21.5,0.0,0.0,0.0,0,7,18,2,19,0


In [3]:
joinColumn = pd.read_csv("../data/data3.csv")
joinColumn

,Unnamed: 0,round,venue,gf,ga,xg,xga,poss,formation,sh,...,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,home_code,target_away,target_draw,target_home
0,0,33,0,0.0,0.0,1.4,0.8,49.0,4,11.0,...,17.333333,6.333333,17.300000,0.333333,0.0,0.0,0,0,1,0
1,1,35,1,2.0,0.0,2.4,0.4,64.0,4,25.0,...,12.000000,3.666667,16.566667,0.333333,0.0,0.0,0,0,0,1
2,2,37,0,5.0,2.0,3.5,1.0,67.0,4,30.0,...,17.666667,6.333333,17.166667,1.000000,0.0,0.0,0,0,0,1
3,3,2,0,1.0,3.0,0.3,1.8,43.0,4,6.0,...,22.000000,8.000000,17.500000,1.333333,0.0,0.0,0,1,0,0
4,4,3,1,2.0,0.0,2.1,0.7,43.0,4,14.0,...,20.333333,8.333333,18.500000,1.666667,0.0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,2953,19,0,1.0,2.0,1.1,1.8,57.0,12,7.0,...,6.666667,2.666667,16.700000,0.000000,0.0,0.0,19,1,0,0
2954,2954,20,1,2.0,4.0,2.7,1.6,65.0,15,23.0,...,6.333333,3.333333,17.166667,0.333333,0.0,0.0,19,1,0,0
2955,2955,21,0,1.0,1.0,0.5,1.3,71.0,12,8.0,...,12.666667,5.666667,18.866667,0.666667,0.0,0.0,19,0,1,0
2956,2956,23,1,1.0,0.0,0.6,1.1,38.0,17,9.0,...,12.666667,4.333333,17.600000,0.666667,0.0,0.0,19,0,0,1


In [4]:
joinTrain = joinColumn[joinColumn["season"] < 2021]
joinTest = joinColumn[joinColumn["season"] > 2021]

In [5]:
joinTrain = joinTrain.dropna()
joinTrain

,Unnamed: 0,round,venue,gf,ga,xg,xga,poss,formation,sh,...,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,home_code,target_away,target_draw,target_home
0,0,33,0,0.0,0.0,1.4,0.8,49.0,4,11.0,...,17.333333,6.333333,17.300000,0.333333,0.0,0.0,0,0,1,0
1,1,35,1,2.0,0.0,2.4,0.4,64.0,4,25.0,...,12.000000,3.666667,16.566667,0.333333,0.0,0.0,0,0,0,1
2,2,37,0,5.0,2.0,3.5,1.0,67.0,4,30.0,...,17.666667,6.333333,17.166667,1.000000,0.0,0.0,0,0,0,1
3,3,2,0,1.0,3.0,0.3,1.8,43.0,4,6.0,...,22.000000,8.000000,17.500000,1.333333,0.0,0.0,0,1,0,0
4,4,3,1,2.0,0.0,2.1,0.7,43.0,4,14.0,...,20.333333,8.333333,18.500000,1.666667,0.0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890,2890,24,0,1.0,0.0,0.5,0.9,50.0,12,7.0,...,13.333333,4.000000,16.033333,0.666667,0.0,0.0,19,0,0,1
2891,2891,28,1,2.0,1.0,1.1,0.3,61.0,12,16.0,...,12.333333,4.333333,15.200000,0.666667,0.0,0.0,19,0,0,1
2892,2892,29,0,0.0,2.0,0.9,3.2,38.0,12,15.0,...,14.333333,4.666667,17.000000,0.666667,0.0,0.0,19,1,0,0
2893,2893,30,0,0.0,1.0,0.3,0.7,59.0,3,9.0,...,12.666667,4.000000,17.666667,0.666667,0.0,0.0,19,1,0,0


In [6]:
joinColumnTrain = joinTrain.loc[:, ['target_away', 'target_draw']]
joinColumnTest = joinTest.loc[:, ['target_away', 'target_draw']]

In [7]:
joinColumnTrain

,target_away,target_draw
0,0,1
1,0,0
2,0,0
3,1,0
4,0,0
...,...,...
2890,0,0
2891,0,0
2892,1,0
2893,1,0


In [8]:
joinColumnTest

,target_away,target_draw
147,0,0
148,0,0
149,0,0
150,0,0
151,0,0
...,...,...
2953,1,0
2954,1,0
2955,0,1
2956,0,0


In [16]:
joinColumnTrain.reset_index(drop=True)
joinColumnTrain

,target_away,target_draw
0,0,1
1,0,0
2,0,0
3,1,0
4,0,0
...,...,...
2890,0,0
2891,0,0
2892,1,0
2893,1,0


In [20]:
newTrain = pd.merge(dataTrain, joinColumnTrain,left_index=True, right_index=True)
newTest = pd.merge(dataTest, joinColumnTest,left_index=True, right_index=True)

In [22]:
joinColumnTrain.to_csv("../data/joinTrain.csv")
joinColumnTest.to_csv("../data/joinTest.csv")

In [3]:
train = pd.read_csv("../data/newtrain.csv")
test = pd.read_csv("../data/newtest.csv")

In [4]:
del train["Unnamed: 0"]
del test["Unnamed: 0"]
del train["target_away"]
del test["target_away"]

del train["gf_rolling"]
del test["gf_rolling"]

del train["ga_rolling"]
del test["ga_rolling"]

del train["sh_rolling"]
del test["sh_rolling"]

del train["sot_rolling"]
del test["sot_rolling"]

del train["dist_rolling"]
del test["dist_rolling"]

del train["fk_rolling"]
del test["fk_rolling"]

del train["pk_rolling"]
del test["pk_rolling"]

del train["pkatt_rolling"]
del test["pkatt_rolling"]
train

,round,venue,gf,ga,xg,xga,poss,formation,sh,sot,dist,fk,pk,pkatt,venue_code,opp_code,hour,day_code,home_code,target_home
0,33,0,0.0,0.0,1.4,0.8,49.0,4,11.0,2.0,17.5,0.0,0.0,0.0,0,5,13,6,0,0
1,35,1,2.0,0.0,2.4,0.4,64.0,4,25.0,8.0,18.4,2.0,0.0,0.0,1,4,12,6,0,1
2,37,0,5.0,2.0,3.5,1.0,67.0,4,30.0,14.0,16.6,2.0,0.0,0.0,0,7,13,6,0,1
3,2,0,1.0,3.0,0.3,1.8,43.0,4,6.0,3.0,20.5,1.0,0.0,0.0,0,12,14,6,0,0
4,3,1,2.0,0.0,2.1,0.7,43.0,4,14.0,5.0,16.4,1.0,0.0,0.0,1,17,12,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,24,0,1.0,0.0,0.5,0.9,50.0,12,7.0,3.0,14.9,1.0,0.0,0.0,0,8,20,4,19,1
1802,28,1,2.0,1.0,1.1,0.3,61.0,12,16.0,6.0,20.5,1.0,0.0,0.0,1,4,12,6,19,1
1803,29,0,0.0,2.0,0.9,3.2,38.0,12,15.0,3.0,17.6,0.0,0.0,0.0,0,17,14,6,19,0
1804,30,0,0.0,1.0,0.3,0.7,59.0,3,9.0,3.0,21.5,0.0,0.0,0.0,0,7,18,2,19,0


In [5]:
subsetTrain = pd.read_csv("../data/joinTrain.csv")
subsetTest = pd.read_csv("../data/joinTest.csv")
subsetTrain

,Unnamed: 0,target_away,target_draw
0,0,0,1
1,1,0,0
2,2,0,0
3,3,1,0
4,4,0,0
...,...,...,...
1801,2890,0,0
1802,2891,0,0
1803,2892,1,0
1804,2893,1,0


In [6]:
mergeTrain = pd.merge(train, subsetTrain, left_index=True, right_index=True)
mergeTest = pd.merge(test, subsetTest, left_index=True, right_index=True)

In [7]:
mergeTrain

,round,venue,gf,ga,xg,xga,poss,formation,sh,sot,...,pkatt,venue_code,opp_code,hour,day_code,home_code,target_home,Unnamed: 0,target_away,target_draw
0,33,0,0.0,0.0,1.4,0.8,49.0,4,11.0,2.0,...,0.0,0,5,13,6,0,0,0,0,1
1,35,1,2.0,0.0,2.4,0.4,64.0,4,25.0,8.0,...,0.0,1,4,12,6,0,1,1,0,0
2,37,0,5.0,2.0,3.5,1.0,67.0,4,30.0,14.0,...,0.0,0,7,13,6,0,1,2,0,0
3,2,0,1.0,3.0,0.3,1.8,43.0,4,6.0,3.0,...,0.0,0,12,14,6,0,0,3,1,0
4,3,1,2.0,0.0,2.1,0.7,43.0,4,14.0,5.0,...,0.0,1,17,12,5,0,1,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,24,0,1.0,0.0,0.5,0.9,50.0,12,7.0,3.0,...,0.0,0,8,20,4,19,1,2890,0,0
1802,28,1,2.0,1.0,1.1,0.3,61.0,12,16.0,6.0,...,0.0,1,4,12,6,19,1,2891,0,0
1803,29,0,0.0,2.0,0.9,3.2,38.0,12,15.0,3.0,...,0.0,0,17,14,6,19,0,2892,1,0
1804,30,0,0.0,1.0,0.3,0.7,59.0,3,9.0,3.0,...,0.0,0,7,18,2,19,0,2893,1,0


In [8]:
del mergeTrain["Unnamed: 0"]
del mergeTest["Unnamed: 0"]

In [9]:
mergeTrain

,round,venue,gf,ga,xg,xga,poss,formation,sh,sot,...,pk,pkatt,venue_code,opp_code,hour,day_code,home_code,target_home,target_away,target_draw
0,33,0,0.0,0.0,1.4,0.8,49.0,4,11.0,2.0,...,0.0,0.0,0,5,13,6,0,0,0,1
1,35,1,2.0,0.0,2.4,0.4,64.0,4,25.0,8.0,...,0.0,0.0,1,4,12,6,0,1,0,0
2,37,0,5.0,2.0,3.5,1.0,67.0,4,30.0,14.0,...,0.0,0.0,0,7,13,6,0,1,0,0
3,2,0,1.0,3.0,0.3,1.8,43.0,4,6.0,3.0,...,0.0,0.0,0,12,14,6,0,0,1,0
4,3,1,2.0,0.0,2.1,0.7,43.0,4,14.0,5.0,...,0.0,0.0,1,17,12,5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,24,0,1.0,0.0,0.5,0.9,50.0,12,7.0,3.0,...,0.0,0.0,0,8,20,4,19,1,0,0
1802,28,1,2.0,1.0,1.1,0.3,61.0,12,16.0,6.0,...,0.0,0.0,1,4,12,6,19,1,0,0
1803,29,0,0.0,2.0,0.9,3.2,38.0,12,15.0,3.0,...,0.0,0.0,0,17,14,6,19,0,1,0
1804,30,0,0.0,1.0,0.3,0.7,59.0,3,9.0,3.0,...,0.0,0.0,0,7,18,2,19,0,1,0


In [10]:
del mergeTrain["venue"]
del mergeTest["venue"]

In [11]:
mergeTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   round        1806 non-null   int64  
 1   gf           1806 non-null   float64
 2   ga           1806 non-null   float64
 3   xg           1806 non-null   float64
 4   xga          1806 non-null   float64
 5   poss         1806 non-null   float64
 6   formation    1806 non-null   int64  
 7   sh           1806 non-null   float64
 8   sot          1806 non-null   float64
 9   dist         1806 non-null   float64
 10  fk           1806 non-null   float64
 11  pk           1806 non-null   float64
 12  pkatt        1806 non-null   float64
 13  venue_code   1806 non-null   int64  
 14  opp_code     1806 non-null   int64  
 15  hour         1806 non-null   int64  
 16  day_code     1806 non-null   int64  
 17  home_code    1806 non-null   int64  
 18  target_home  1806 non-null   int64  
 19  target

In [12]:
mergeTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   round        1806 non-null   int64  
 1   gf           1806 non-null   float64
 2   ga           1806 non-null   float64
 3   xg           1806 non-null   float64
 4   xga          1806 non-null   float64
 5   poss         1806 non-null   float64
 6   formation    1806 non-null   int64  
 7   sh           1806 non-null   float64
 8   sot          1806 non-null   float64
 9   dist         1806 non-null   float64
 10  fk           1806 non-null   float64
 11  pk           1806 non-null   float64
 12  pkatt        1806 non-null   float64
 13  venue_code   1806 non-null   int64  
 14  opp_code     1806 non-null   int64  
 15  hour         1806 non-null   int64  
 16  day_code     1806 non-null   int64  
 17  home_code    1806 non-null   int64  
 18  target_home  1806 non-null   int64  
 19  target

In [13]:
mergeTrain.to_csv("../data/mergeTrain.csv")
mergeTest.to_csv("../data/mergeTest.csv")